### Conclusion

That's it for this example, have fun!

In [1]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Controller()

In [2]:

from jetbot import Robot
import traitlets
import time
robot = Robot()
def forward(change):
        robot.forward(0.4)
controller.buttons[0].observe(forward,names='value')
def backward(change):
        robot.backward(0.4)
controller.buttons[2].observe(backward,names='value')
def right(change):
        robot.right(0.2)
        time.sleep(0.1)
        robot.stop()
controller.buttons[1].observe(right,names='value')
def left(change):
        robot.left(0.2)
        time.sleep(0.1)
        robot.stop()
controller.buttons[3].observe(left,names='value')
def Stop(change):
        robot.stop()
controller.buttons[5].observe(Stop,names='value')

image = widgets.Image(format='jpeg', width=300, height=300)
#display(image)

from jetbot import Camera
camera = Camera.instance()

from jetbot import bgr8_to_jpeg
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

from jetbot import Heartbeat
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        left_link.unlink()
        right_link.unlink()
        robot.stop()
heartbeat = Heartbeat(period=0.5)
 #attach the callback function to heartbeat status
heartbeat.observe(handle_heartbeat_status, names='status')

# only call this if your robot links were unlinked, otherwise we'll have redundant links which will double
# the commands transferred
#left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
#right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

import uuid
import subprocess
subprocess.call(['mkdir', '-p', 'snapshots'])
snapshot_image = widgets.Image(format='jpeg', width=300, height=300)
def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value
controller.buttons[4].observe(save_snapshot, names='value')
display(widgets.HBox([image, snapshot_image]))
display(controller)
#camera.stop()

Controller(axes=(Axis(value=0.003921627998352051), Axis(value=0.003921627998352051), Axis(value=0.003921627998…

In [ ]:
camera.stop()